In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.model_selection import train_test_split


In [51]:
target_1 = []
target_2 = []
target_3 = []

surface_1 = []
surface_2 = []
surface_3 = []
surfaces = []


In [52]:
df = pd.read_csv('/home/jayanaka_98/Documents/AI Model Data/Processed Data/S1/S1_1.csv', header=0)
df.drop(['Acceleration','F*x area'], axis=1)

,Position,Reaction Force,Velocity,Stiffness
0,0.000029,0.052305,0.000000,-26312.355307
1,0.000029,0.050979,0.002891,-26279.210486
2,0.000029,0.049715,0.002882,-26247.591568
3,0.000029,0.048432,0.002873,-26215.508089
4,0.000029,0.047170,0.002863,-26183.951925
...,...,...,...,...
29995,0.000024,-0.020274,0.000734,-21775.342407
29996,0.000024,-0.019678,0.000731,-21788.598176
29997,0.000024,-0.019155,0.000729,-21800.209830
29998,0.000024,-0.018657,0.000727,-21811.285629


In [53]:
n = 10
for i in range(1, n):  #107
    filename = '/home/jayanaka_98/Documents/AI Model Data/Processed Data/S1/S1_' + str(i) +'.csv'
    df = pd.read_csv(filename, header=0)
    df = df.drop(['Acceleration','F*x area'], axis=1)
    surface_1.append(df.values)
    target_1.append(0)
for i in range(1, n):  #107
    filename = '/home/jayanaka_98/Documents/AI Model Data/Processed Data/S2/S2_' + str(i) +'.csv'
    df = pd.read_csv(filename, header=0)
    df = df.drop(['Acceleration','F*x area'], axis=1)
    surface_2.append(df.values)
    target_2.append(1)
for i in range(1, n):  #107
    filename = '/home/jayanaka_98/Documents/AI Model Data/Processed Data/S3/S3_' + str(i) +'.csv'
    df = pd.read_csv(filename, header=0)
    df = df.drop(['Acceleration','F*x area'], axis=1)
    surface_3.append(df.values)
    target_3.append(2)

In [54]:
# extract the time series data and the corresponding labels
X1 = np.array(surface_1)
X1 = X1.reshape(X1.shape[0], -1)

X2 = np.array(surface_2)
X2 = X2.reshape(X2.shape[0], -1)

X3 = np.array(surface_3)
X3 = X3.reshape(X3.shape[0], -1)

y1 = np.array(target_1)
y2 = np.array(target_2)
y3 = np.array(target_3)

In [55]:
target_2

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [56]:
X1_train, X1_test, y1_train, y1_test = train_test_split(surface_1, target_1, test_size=0.2, random_state=20)
X2_train, X2_test, y2_train, y2_test = train_test_split(surface_2, target_2, test_size=0.2, random_state=20)
X3_train, X3_test, y3_train, y3_test = train_test_split(surface_3, target_3, test_size=0.2, random_state=20)

x_test = np.concatenate((X1_test,X2_test,X3_test), axis=0)
x_train = np.concatenate((X1_train,X2_train,X3_train), axis=0)
y_test = np.concatenate((y1_test,y2_test,y3_test), axis=0)
y_train = np.concatenate((y1_train,y2_train,y3_train), axis=0)

In [57]:
model = models.Sequential()
model.add(layers.Conv1D(4, 2, activation='relu', input_shape=(30000, 4)))
model.add(layers.MaxPooling1D(5))
model.add(layers.Flatten())
model.add(layers.Dense(23996, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

2023-05-21 18:32:06.793743: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (mklcpu) ran out of memory trying to allocate 2.14GiB (rounded to 2303232256)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-21 18:32:06.793782: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for mklcpu
2023-05-21 18:32:06.793789: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-05-21 18:32:06.793809: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-05-21 18:32:06.793815: I tensorflow/tsl/framework/bfc_allocator.cc:1046

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[23996,23996] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2]

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=5, validation_data=(x_test, y_test))

Epoch 1/10


2023-05-21 18:28:42.591125: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at sparse_xent_op.cc:102 : INVALID_ARGUMENT: Received a label value of 3 which is outside the valid range of [0, 3).  Label values: 2 2 3 1 3
2023-05-21 18:28:42.591167: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Received a label value of 3 which is outside the valid range of [0, 3).  Label values: 2 2 3 1 3
	 [[{{node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/jayanaka_98/anaconda3/envs/FYP/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jayanaka_98/anaconda3/envs/FYP/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/jayanaka_98/anaconda3/envs/FYP/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/jayanaka_98/anaconda3/envs/FYP/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/jayanaka_98/anaconda3/envs/FYP/lib/python3.11/asyn

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')